In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from typing import List
from langchain_community.document_loaders import PyPDFLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import faiss
from typing import List
from langchain_core.runnables.config import run_in_executor
import os
from typing import Any, Dict, Iterator, List, Mapping, Optional
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [30]:
tokenizer = AutoTokenizer.from_pretrained("MBZUAI/LaMini-T5-738M")
model = AutoModelForSeq2SeqLM.from_pretrained("MBZUAI/LaMini-T5-738M").to(device)

In [10]:
input_prompt = 'based on the given information: "Parameter-Efficient Tuning Large Language Models for Graph\nRepresentation Learning\nQi Zhu\nAmazon Web Services\nqzhuamzn@amazon.com\nDa Zheng\nAmazon Web Services\ndzzhen@amazon.com\nXiang Song\nAmazon Web Services\ndzzhen@amazon.com\nShichang Zhang∗\nUniversity of California, Los Angeles\nshichang@cs.ucla.edu\nBowen Jin∗\nUniversity of Illinois\nUrbana-Champaign\nbowenj4@illinois.edu\nYizhou Sun\nUniversity of California, Los Angeles\nyzsun@cs.ucla.edu\nGeorge Karypis\nAmazon Web Services\ngkarypis@amazon.com\nABSTRACT\nText-rich graphs, which exhibit rich textual information on nodes\nand edges, are prevalent across a wide range of real-world business\napplications. Large Language Models (LLMs) have demonstrated re-\nmarkable abilities in understanding text, which also introduced the\npotential for more expressive modeling in text-rich graphs. Despite\nthese capabilities, efficiently applying LLMs to representation learn-\ning on graphs presents significant challenges. Recently, parameter-\nefficient fine-tuning methods for LLMs have enabled efficient new\ntask generalization with minimal time and memory consumption.\nInspired by this, we introduce Graph-aware Parameter-Efficient\nFine-Tuning - GPEFT, a novel approach for efficient graph repre-\nsentation learning with LLMs on text-rich graphs. Specifically, we\nutilize a graph neural network (GNN) to encode structural infor-\nmation from neighboring nodes into a graph prompt. This prompt\nis then inserted at the beginning of the text sequence. To improve\nthe quality of graph prompts, we pre-trained the GNN to assist the\nfrozen LLM in predicting the next token in the node text. Compared\nwith existing joint GNN and LMs, our method directly generates the\nnode embeddings from large language models with an affordable\nfine-tuning cost. We validate our approach through comprehen-\nsive experiments conducted on 8 different text-rich graphs, observ-\ning an average improvement of 2% in hit@1 and Mean Reciprocal\nRank (MRR) in link prediction evaluations. Our results demonstrate\nthe efficacy and efficiency of our model, showing that it can be\nsmoothly integrated with various large language models, including\nOPT, LLaMA, and Falcon.\n∗Work done while being an intern at Amazon.\nPermission to make digital or hard copies of all or part of this work for personal or\nclassroom use is granted without fee provided that copies are not made or distributed\nfor profit or commercial advantage and that copies bear this notice and the full citation\non the first page. Copyrights for components of this work owned by others than the\nauthor(s) must be honored. Abstracting with credit is permitted. To copy otherwise, or\nrepublish, to post on servers or to redistribute to lists, requires prior specific permission\nand/or a fee. Request permissions from permissions@acm.org.\nConference’17, July 2017, Washington, DC, USA\n©2024 Copyright held by the owner/author(s). Publication rights licensed to ACM.\nACM ISBN 978-x-xxxx-xxxx-x/YY/MM\nhttps://doi.org/10.1145/nnnnnnn.nnnnnnn\nKEYWORDS\nRepresentation Learning, Graphs, Language Models\nACM Reference Format:\nQi Zhu, Da Zheng, Xiang Song, Shichang Zhang, Bowen Jin, Yizhou Sun,\nand George Karypis. 2024. Parameter-Efficient Tuning Large Language\nModels for Graph Representation Learning. In ACM, New York, NY, USA,\n11 pages. https://doi.org/10.1145/nnnnnnn.nnnnnnn\n1 INTRODUCTION\nInformation networks form the backbone of modern data systems:\nmillions of daily social posts are shared among users on platforms\nlike Facebook and Twitter; countless papers are published and cited\nwithin academic networks. Many of these networks are rich in\ntextual information on various types of objects, known as text-rich\nor text-attributed networks. For example, in an e-commerce graph,\ntext-rich information could be used to predict links by analyzing\nproduct descriptions, user reviews, and metadata to recommend\nitems that are frequently bought together. Such real-world applica-\ntions typify the problem of link prediction, where representation\nlearning (i.e., embedding) emerges as the most prevalent solution.\nRepresentation learning on graph-structured data aims to learn\na dense vector for each node through self-supervised tasks such as\nlink prediction [9,34,46] or masked attribute recovery [14]. These\ndense vectors can be widely utilized in various downstream ap-\nplications including search, ranking, and retrieval. To effectively\nutilize both node attributes and structural information in represen-\ntation learning, Graph Neural Networks (GNNs) [11,21] devise a\nnovel type of neural network with a message-passing mechanism.\nIn text-rich graphs, raw text is usually transformed into feature\nvectors by a text encoder prior to the application of Graph Neu-\nral Networks (GNNs). Early text encoders, such as bag-of-words\nand Word2Vec [32], are being gradually phased out due to signifi-\ncant improvements in text representation from transformer-based\nlanguage models (LMs).\nThis has spurred interest in jointly modeling textual and struc-\ntural data with GNNs and LMs. Notable architectures include (1)\na cascading architecture that integrates GNNs with LM features\n(cascading GNN-LMs); (2) nested GNN-LMs that blend message\npassing into the language modeling process. Examples of the latter\ninclude arXiv:2404.18271v1 [cs.CL] 28 Apr 2024"\
". answer this question: what is the purpose of this research paper?'

In [22]:
rag = """ You are a helpful assistant who tries to answer the question at the end using this information:  'Conference’17, July 2017, Washington, DC, USA Qi Zhu, Da Zheng, Xiang Song, Shichang Zhang, Bowen Jin, Yizhou Sun, and George Karypis\n(2) In the same tables, we also find GNN (Sentence-BERT) per-\nform worse than GNN (PEFT-LLaMA). In addition, GraphForm-\ners also reports subpar performance against GNN (PEFT-LLaMA).\nTherefore, we conclude that large language models serve as power-\nful feature encoders for representation learning on text-rich graphs.\nMeanwhile, InstructGLM outperforms GraphFormers but falls short\nof PATTON’s performance. This discrepancy may be attributed to\nthe challenges of handling graph structures through embedding-\nbased in-context learning, which struggles with accurately cap-\nturing structures via natural language descriptions. Consequently,\ncarefully fine-tuned GNN-LMs ( GPEFT and PATTON) tend to sur-\npass the embeddings produced by black-box LLMs.\n(3)GPEFT consistently outperforms the compared baseline across\nboth metrics. Specifically, it surpasses PEFT-LLaMA by 2.6% and\n1.8% on MAG and Amazon Reviews, respectively, in terms of hit@1.\nCompared to its own ablation, we find that pre-training helps im-\nprove the accuracy of link prediction, especially in the academic\ntext-rich graph.\nRQ2: Is the pre-training of the graph prompt encoder neces-\nsary?\nOn MAG, we observe that our approach performs significantly\nworse (∼6%) without pretraining compared to its pretrained coun-\nterpart. On Amazon Review, while the performance gap is smaller\nacross three subdomains, the Video Games category demonstrates\ninstability without pretraining. Interestingly, Patton also outper-\nforms GraphFormers with the same architecture on subdomains\nthat utilize pre-trained checkpoints. Both methods introduce virtual\nstructural tokens, and employing the same pre-training objective\nas the pre-trained language model. Apparently, continual training\nhelps align the virtual token representations with text representa-\ntions. Therefore, we advocate for a pre-training then fine-tuning\nparadigm, where one needs to pre-train only once in a domain and\ncan fine-tune for various applications ( e.g., co-purchase, churn, etc.).\n5.5 Model Analysis\nIn this section, we provide more in-depth study to understand\nperformance and efficiency of GPEFT .\nRQ3:Can our approach be adapted to different PEFT methods\nand LLMs?\n(a) Varying PEFT algorithms\n (b) Varying backbone LLMs\nFigure 3: GPEFT using different LLMs and PEFT algorithms\non Amazon Review.\nWe first apply prefix tuning [ 24] onGPEFT and demonstrate the\nresults on four subdomains of Amazon Review in Figure 3a. We\ncan find that using prefix-tuning or LORA does not show muchTable 5: Time and memory costs of GPEFT on MAG-\nEconomics using 8 A100 GPUs with a total batch size of 32.\npre-training fine-tuning inference\nTime 36min 96min 11min\nMemory 20883MB 28309MB 13676MB\n#trainable 5.5M 14.4M -\ndifference, which indicates the versatility of our framework on\nvarious PEFT techniques.\nSecond, we substitute the backbone language model with OPT-\n1.3B [ 47] and Falcon-7B2. In Figure 3b, we observe that, in general,\nLLaMA-7B outperforms the others, with Falcon-7B in second place\nand OPT-1.3B being outperformed by these two models. Our re-\nsults are consistent with other work that shows LLMs with more\nparameters yield better performance in downstream applications.\nRQ4: Is GPEFT efficient in large-scale representation learn-\ning?\nIn Table 5, we report the running time of our approach at each\nphase on MAG-Economics. Specifically, unlike the pre-training\ntime of PATTON [ 19] (10hr+), the pre-training time of GPEFT is\neven shorter than the fine-tuning time. Because we only optimize\nΘ𝑔in the pre-training phase and we can say that the pre-training\nis both necessary and efficient together with RQ2 . Owing to its\nparameter-efficient tuning, our approach not only minimizes the\nnumber of training parameters but also achieves the best perfor-\nmance. As mentioned in the introduction, the design of GPEFT\naims to provide a high-quality, lightweight framework suitable for\nvarious applications on different text-rich graphs. Considering the\nreduced parameter storage requirements and the training time, we\nbelieve GPEFT holds significant potential for industrial-scale ap-\nplications, including recommendation systems, ranking tasks, and\netc.\nRQ5: What is the parameter sensitivity of GPEFT ?\nThere are several important hyper-parameters in GPEFT : (1)\nLORA rank 𝑟, it affects the number of trainable parameters (2)\nnumber of hops 𝑘in GNN prompt encoder, it affects the amount of\nstructural information used in the training of GPEFT . In Figure 4,\nwe observe that varying the rank of LoRA matrices from 4 to 32 does\nnot significantly affect the performance. This observation aligns\nwith findings from other studies, which suggest that optimizing for\na single task requires only minor adjustments to the pre-trained\nLLM weights. Similarly, varying number of hops in the GNN prompt\nencoder has a minor effect on performance, until the point where\nmessage passing begins to aggregate more noisy neighbors than\nuseful ones (e.g., at 4 hops, as shown in Figure 4b).\n6 CONCLUSION AND FUTURE WORK\nThis paper proposes GPEFT to harness LLMs for representation\nlearning on text-rich graphs. Compared with existing work of ap-\nplying LLM on graph structure data, our proposed method is the\nfirst one that generates embeddings from the LLM and therefore can\nbe applied on numerous industrial applications. More importantly,\n2https://huggingface.co/tiiuae/falcon-7b', metadata={'source': './prof_1/2404.18271v1.pdf', 'page': 7})]"""

In [31]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="MBZUAI/LaMini-Flan-T5-783M")

config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [32]:
pipe(rag, truncation=True, do_sample=True)[0]['generated_text']

/home/sayyss/.conda/envs/RAG/lib/python3.12/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'The text that provides data for modeling is unanswerable as the provided information is a'

In [18]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v0.4", device=0)

In [23]:
rag += ". answer this question: what is the purpose of this research paper?"

In [25]:
pipe(rag, truncation=True, do_sample=True)[0]['generated_text']

" You are a helpful assistant who tries to answer the question at the end using this information:  'Conference’17, July 2017, Washington, DC, USA Qi Zhu, Da Zheng, Xiang Song, Shichang Zhang, Bowen Jin, Yizhou Sun, and George Karypis\n(2) In the same tables, we also find GNN (Sentence-BERT) per-\nform worse than GNN (PEFT-LLaMA). In addition, GraphForm-\ners also reports subpar performance against GNN (PEFT-LLaMA).\nTherefore, we conclude that large language models serve as power-\nful feature encoders for representation learning on text-rich graphs.\nMeanwhile, InstructGLM outperforms GraphFormers but falls short\nof PATTON’s performance. This discrepancy may be attributed to\nthe challenges of handling graph structures through embedding-\nbased in-context learning, which struggles with accurately cap-\nturing structures via natural language descriptions. Consequently,\ncarefully fine-tuned GNN-LMs ( GPEFT and PATTON) tend to sur-\npass the embeddings produced by black-box LLMs.\n(3)G

In [5]:
from transformers import pipeline

pipe = pipeline("text-generation", model="MBZUAI/LaMini-Neo-1.3B")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [27]:
pipe(rag, max_length=1500, truncation=True, do_sample=True)[0]['generated_text']

Both `max_new_tokens` (=32) and `max_length`(=1500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


" You are a helpful assistant who tries to answer the question at the end using this information:  'Conference’17, July 2017, Washington, DC, USA Qi Zhu, Da Zheng, Xiang Song, Shichang Zhang, Bowen Jin, Yizhou Sun, and George Karypis\n(2) In the same tables, we also find GNN (Sentence-BERT) per-\nform worse than GNN (PEFT-LLaMA). In addition, GraphForm-\ners also reports subpar performance against GNN (PEFT-LLaMA).\nTherefore, we conclude that large language models serve as power-\nful feature encoders for representation learning on text-rich graphs.\nMeanwhile, InstructGLM outperforms GraphFormers but falls short\nof PATTON’s performance. This discrepancy may be attributed to\nthe challenges of handling graph structures through embedding-\nbased in-context learning, which struggles with accurately cap-\nturing structures via natural language descriptions. Consequently,\ncarefully fine-tuned GNN-LMs ( GPEFT and PATTON) tend to sur-\npass the embeddings produced by black-box LLMs.\n(3)G

In [13]:
from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional
from pydantic import Field

class T5Embeddings(Embeddings):

    def __init__(self, model_name: str = "MBZUAI/LaMini-T5-738M"):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)

    def _embed_text(self, text: str) -> List[float]:
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        input_ids = inputs.input_ids.to(self.device)
        
        with torch.no_grad():
            embeddings = self.model.shared(input_ids).mean(dim=1)
        
        return embeddings.squeeze().cpu().numpy().tolist()
        
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed search docs."""
        return [self._embed_text(text) for text in texts]


    def embed_query(self, text: str) -> List[float]:
        """Embed query text."""
        return self._embed_text(text)

class CustomLLM(LLM):
    n: int
    device = "cuda" if torch.cuda.is_available() else "cpu"
    """The number of characters from the last message of the prompt to be echoed."""

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:

        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
            
        input_ids = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
        input_ids = input_ids.input_ids.to(self.device)
        
        outputs = model.generate(input_ids, max_length=512)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters."""
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": "CustomChatModel",
        }

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "custom"

In [27]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v0.4", device=0)

class CustomLLM_tiny(LLM):
    n: int
    device = "cuda" if torch.cuda.is_available() else "cpu"
    """The number of characters from the last message of the prompt to be echoed."""

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:

        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")\

        return pipe(prompt, do_sample=True)[0]['generated_text']

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters."""
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": "CustomChatModel",
        }

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "custom"

In [28]:
def get_embeddings_from_pdf(pdf_folder_path, embeddings):
    documents = []
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())
            
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
    chunked_documents = text_splitter.split_documents(documents)
    faiss_index = FAISS.from_documents(chunked_documents, embeddings)
    
    return faiss_index


### TinyLlama with llama embeddings

In [29]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

llm = CustomLLM_tiny(n=5)
embeddings = (
    OllamaEmbeddings()
)

faiss_index = get_embeddings_from_pdf("./prof_1/", embeddings)
retriever = faiss_index.as_retriever()

# Create a prompt template
template = """You are an helpful assistant and you try to use the following information to answer the questions given at the end:  
Context: {context}

Question: {question}

Answer:"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)


In [32]:

query = "what is a graph neural network (GNN)?"
result = qa_chain({"queary": query})
print(result['result'])

You are an helpful assistant and you try to use the following information to answer the questions given at the end:  
Context: Conference’17, July 2017, Washington, DC, USA Qi Zhu, Da Zheng, Xiang Song, Shichang Zhang, Bowen Jin, Yizhou Sun, and George Karypis
Neural Information Processing Systems 33 (2020), 16857–16867.
[41] Yijun Tian, Huan Song, Zichen Wang, Haozhu Wang, Ziqing Hu, Fang Wang,
Nitesh V Chawla, and Panpan Xu. 2023. Graph Neural Prompting with Large
Language Models. arXiv preprint arXiv:2309.15427 (2023).
[42] Hugo Touvron, Thibaut Lavril, Gautier Izacard, Xavier Martinet, Marie-Anne
Lachaux, Timothée Lacroix, Baptiste Rozière, Naman Goyal, Eric Hambro, Faisal
Azhar, et al .2023. Llama: Open and efficient foundation language models. arXiv
preprint arXiv:2302.13971 (2023).
[43] Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones,
Aidan N Gomez, Lukasz Kaiser, and Illia Polosukhin. 2017. Attention is all
you need. Advances inneural information process

### T5 model with T5 embeddings

In [24]:
embeddings = T5Embeddings("MBZUAI/LaMini-T5-738M") 
faiss_index = get_embeddings_from_pdf("./prof_1/", embeddings)
retriever = faiss_index.as_retriever(search_kwargs={"k": 1})
llm = CustomLLM(n=5)

# Create a prompt template
template = """You are an helpful assistant and you try to use the following information to answer the questions given at the end:  
Context: {context}

Question: {question}

Answer:"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
embeddings = T5Embeddings("MBZUAI/LaMini-T5-738M") 
faiss_index = get_embeddings_from_pdf("./prof_1/", embeddings)
retriever = faiss_index.as_retriever(search_kwargs={"k": 1})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
retriever.invoke("how to utilize a graph neural network (GNN) to encode structural infor-"\
"mation from neighboring nodfes into a graph prompt?")

[Document(page_content='Parameter-Efficient Tuning Large Language Models for Graph\nRepresentation Learning\nQi Zhu\nAmazon Web Services\nqzhuamzn@amazon.comDa Zheng\nAmazon Web Services\ndzzhen@amazon.comXiang Song\nAmazon Web Services\ndzzhen@amazon.com\nShichang Zhang∗\nUniversity of California, Los Angeles\nshichang@cs.ucla.eduBowen Jin∗\nUniversity of Illinois\nUrbana-Champaign\nbowenj4@illinois.eduYizhou Sun\nUniversity of California, Los Angeles\nyzsun@cs.ucla.edu\nGeorge Karypis\nAmazon Web Services\ngkarypis@amazon.com\nABSTRACT\nText-rich graphs, which exhibit rich textual information on nodes\nand edges, are prevalent across a wide range of real-world business\napplications. Large Language Models (LLMs) have demonstrated re-\nmarkable abilities in understanding text, which also introduced the\npotential for more expressive modeling in text-rich graphs. Despite\nthese capabilities, efficiently applying LLMs to representation learn-\ning on graphs presents significant challeng

In [25]:

"T5 MODEL WITH T5 EMBEDDINGS" 

query = "how to utilize a graph neural network (GNN) to encode structural infor-"\
"mation from neighboring nodfes into a graph prompt?"

result = qa_chain({"query": query})
print(result['result'])

The text does not provide information about which model is more efficient in predicting the next token in the node text.


### T5 model with llama embeddings

In [16]:

embeddings = (
    OllamaEmbeddings()
)
llm = CustomLLM(n=5)

faiss_index = get_embeddings_from_pdf("./prof_1/", embeddings)
retriever = faiss_index.as_retriever()


# Create a prompt template
template = """You are an helpful assistant and you try to use the following information to answer the questions given at the end:  
Context: {context}

Question: {question}

Answer:"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:

"T5 MODEL WITH LLAMA EMBEDDINGS"

query = "how to utilize a graph neural network (GNN) to encode structural infor-"\
"mation from neighboring nodes into a graph prompt?"

result = qa_chain({"query": query})
print(result['result'])

In [18]:
faiss_index = get_embeddings_from_pdf("./prof_1/", embeddings)
retriever = faiss_index.as_retriever(search_kwargs={"k": 1})


In [19]:
retriever.invoke("what is a GNN")

[Document(page_content='Conference’17, July 2017, Washington, DC, USA Qi Zhu, Da Zheng, Xiang Song, Shichang Zhang, Bowen Jin, Yizhou Sun, and George Karypis\n(2) In the same tables, we also find GNN (Sentence-BERT) per-\nform worse than GNN (PEFT-LLaMA). In addition, GraphForm-\ners also reports subpar performance against GNN (PEFT-LLaMA).\nTherefore, we conclude that large language models serve as power-\nful feature encoders for representation learning on text-rich graphs.\nMeanwhile, InstructGLM outperforms GraphFormers but falls short\nof PATTON’s performance. This discrepancy may be attributed to\nthe challenges of handling graph structures through embedding-\nbased in-context learning, which struggles with accurately cap-\nturing structures via natural language descriptions. Consequently,\ncarefully fine-tuned GNN-LMs ( GPEFT and PATTON) tend to sur-\npass the embeddings produced by black-box LLMs.\n(3)GPEFT consistently outperforms the compared baseline across\nboth metrics. Sp

### Llama 2 with T5 embeddings

In [8]:
llm = Ollama(model="llama2")
embeddings = T5Embeddings("MBZUAI/LaMini-T5-738M")
    
faiss_index = get_embeddings_from_pdf("./prof_1/", embeddings)
retriever = faiss_index.as_retriever()

# Create a prompt template
template = """You are an helpful assistant and you try to use the following information to answer the questions given at the end:  
Context: {context}

Question: {question}

Answer:"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:

"LLAMA 2 7B with T5 embeddings" 

query = "how to utilize a graph neural network (GNN) to encode structural infor-"\
"mation from neighboring nodes into a graph prompt?"

result = qa_chain({"query": query})
print(result['result'])

The paper proposes a novel approach called Graph Prompt Encoder and Fine-tuned LM (GPEFT) that leverages large language models (LLMs) for representation learning on text-rich graphs. The key innovation of GPEFT is the use of a graph neural network (GNN) to encode structural information from neighboring nodes into a graph prompt, which is then fine-tuned with an LLM to learn high-quality representations for downstream tasks.

The proposed approach consists of two main components:

1. Graph Prompt Encoder (GPE): This component is responsible for encoding the structural information from the input graph into a compact representation, called the graph prompt. The GPE uses a GNN to model the relationships between nodes in the graph and generates a set of embeddings that capture the structural information.
2. Fine-tuned LLM (FLLM): This component is responsible for fine-tuning the LLM with the generated graph prompt to learn high-quality representations for downstream tasks. The FLLM uses the

### Llama 2 with llama embeddings

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

llm = Ollama(model="llama2")
embeddings = (
    OllamaEmbeddings()
)

faiss_index = get_embeddings_from_pdf("./prof_1/", embeddings)
retriever = faiss_index.as_retriever()

# Create a prompt template
template = """You are an helpful assistant and you try to use the following information to answer the questions given at the end:  
Context: {context}

Question: {question}

Answer:"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:

"LLAMA 2 7b with llama embeddings" 

query = "how to utilize a graph neural network (GNN) to encode structural infor-"\
"mation from neighboring nodes into a graph prompt?"

result = qa_chain({"query": query})
print(result['result'])

In [ ]:
# Check t5 instructions finetuning format
# GPT-NeoX 1.3
# Sliding windows

# Shorten retrieve content
# Finetune to extend context
# Try different techniques for FAISS